In [205]:
import numpy as np
from qiskit import QuantumCircuit
import random
%matplotlib inline

Summary
1. Find the qubits who's state isn't used in the next state
2. copy duplicate values onto those states
3. Apply not gates where needed
4. Swap around

In [644]:
def genBoolFuncs(N=5):
    '''K=1 Assumed'''
    funcs = np.empty(N)
    for i in range(N):
        randVal = i
        while abs(randVal) == i: # this function for preventing self loops is flawed
            randVal = random.randrange(1, N+1)
        funcs[i] = randVal if random.random() > 0.5 else -randVal
    
    return funcs

In [645]:
funcs = genBoolFuncs()

In [646]:
print(funcs)

[-4.  3. -1. -5.  1.]


Read Funcs:

Turns function vector into a count of how many time each qubit value is used and which qubits aren't used

In [647]:
def readFuncs(funcs, N=5):
#     arr1 = np.concatenate([np.arange(-N, 0), np.arange(1, N+1)])
    arr1 = np.arange(-N, N + 1)
    arr2 = funcs
    idx = np.searchsorted(ideal, funcs)
    idx[idx==len(arr1)] = 0
    mask = arr1[idx]==arr2
    out = np.bincount(idx[mask])
    full_counts = np.pad(out, (0, arr1.size - out.size), 'constant')
    counts = np.array([full_counts[N - 1 - i] + full_counts[full_counts.size - 1 - (N - 1 - i)] for i in range(N)])
    unused = np.where(counts==0)[0]
    return full_counts, counts, unused

In [648]:
full_counts, counts, unused = readFuncs(funcs)
print(full_counts)
print(counts)
print(unused)

[1 1 0 0 1 0 1 0 1 0 0]
[2 0 1 1 1]
[1]


In [649]:
def initCircuit(N=5):
    qc = QuantumCircuit(N)
    for i in range(N):
        init_state = [0,1] if random.random() > 0.5 else [1, 0]
        qc.initialize(init_state, i)
    return qc

In [650]:
qc = initCircuit()

In [651]:
qc.draw()

┌─────────────────┐
q_0: ┤ initialize(0,1) ├
     ├─────────────────┤
q_1: ┤ initialize(1,0) ├
     ├─────────────────┤
q_2: ┤ initialize(1,0) ├
     ├─────────────────┤
q_3: ┤ initialize(0,1) ├
     ├─────────────────┤
q_4: ┤ initialize(1,0) ├
     └─────────────────┘

In [636]:
def copy_to_qubit(qc, control, unused):
    print("UNUSED: ")
    print(unused)
    unused_qubit = unused[0]
    print("q idx: " + str(unused_qubit))
    qc.reset(unused_qubit) # -1 because unused array is not 0 indexed
    qc.cx(control, unused_qubit)
    unused = np.delete(unused, 0)
    print(unused)
    return unused_qubit, unused

Copying Description:

Let's say we want to get: [-2, -1, -1, 2, -3]
Right now we have: [1, 2, 3, 4 , 5]

What the copying step does is that it searches for all unused qubits (in this case [4,5]), then copies the value of the duplicate qubits (in this case [1, 2, 3]) to those unused qubits depending on how many times we need it (we need 2 copies of 1, 2 copies of 2, 1 of 3)

If a qubit needs more than 1 copy, we set an unused qubit to 0, then CNOT with a duplicate qubit to set the value. we can apply a NOT gate after if we need a negative number (which symbolizes a NOT)

As a result, we can get from [1, 2, 3, 4 , 5] to [-1, 2, 3, -2, -1], which the next step will then sort

In [652]:
def genCopyingCircuit(qc, full_counts, counts, unused, N=5):
    current_values = np. arange(1, N + 1)
    for idx, val in enumerate(counts):
        if val > 1:
            num_negatives = full_counts[counts.size - 1 - idx]
            if val == num_negatives: # if all occurences of an input are used with NOT
                qc.x(idx)
                current_values[idx] *= -1
                for i in range(val - 1):
                    unused_qubit = unused[0]
                    qc.reset(unused_qubit) 
                    qc.cx(idx, unused_qubit)
                    current_values[unused_qubit] = current_values[idx]
                    unused = np.delete(unused, 0)
#                     copy_to_qubit(qc, idx, unused)
            elif val > num_negatives: # if there are some end values that are not under the NOT gate
                for i in range(val - 1):
                    unused_qubit = unused[0]
                    qc.reset(unused_qubit) 
                    qc.cx(idx, unused_qubit)
                    current_values[unused_qubit] = current_values[idx]
                    unused = np.delete(unused, 0)
                    if num_negatives > 0:
                        qc.x(unused_qubit)
                        current_values[unused_qubit] *= -1
                        num_negatives -= 1
            else: # the number of negatives should never be more than the actual 
                raise ValueError()
                
        elif val == 1:
            num_negatives = full_counts[counts.size - 1 - idx]
            if num_negatives == val:
                qc.x(idx)
                current_values[idx] *= -1
                
    return current_values, qc
            
            
                
    

In [653]:
current_vals, qc = genCopyingCircuit(qc, full_counts, counts, unused)
print(current_vals)
qc.draw()

[ 1 -1  3 -4 -5]


┌─────────────────┐               
q_0: ┤ initialize(0,1) ├───────■───────
     ├─────────────────┤     ┌─┴─┐┌───┐
q_1: ┤ initialize(1,0) ├─|0>─┤ X ├┤ X ├
     ├─────────────────┤     └───┘└───┘
q_2: ┤ initialize(1,0) ├───────────────
     ├─────────────────┤┌───┐          
q_3: ┤ initialize(0,1) ├┤ X ├──────────
     ├─────────────────┤├───┤          
q_4: ┤ initialize(1,0) ├┤ X ├──────────
     └─────────────────┘└───┘

Swap Description:

Now we need to reorder our qubit states. 

To do so, we find all incorrect state locations, find where they should be, then execute swaps to get the right value to one place. Then we repeat until there are no more incorrect state locations

In [654]:
def genSwapCircuit(qc, current_vals, funcs, N=5):
    incorrect_positions = np.where(funcs != current_vals)[0]
    print("Current Position: " + np.array2string(current_vals))
    print("Intended Position: " + np.array2string(funcs))
    while (not np.array_equal(incorrect_positions, np.zeros(0))):
        incorrect_pos = incorrect_positions[0]
        possible_moves = np.where(current_vals == funcs[incorrect_pos])[0] # finds where the correct value for the first slot with an incorrect value is
        for i in possible_moves:
            if (np.isin(incorrect_positions, i).any()):#if the move is also in wrong place then make the swap
                qc.swap(incorrect_pos, i)
                current_vals[incorrect_pos], current_vals[i] = current_vals[i], current_vals[incorrect_pos]
                incorrect_positions = np.where(funcs != current_vals)[0] # post swap, check for remaining incorrects
    
    print("Current Position: " + np.array2string(current_vals))
    print("Intended Position: " + np.array2string(funcs))
#     print(np.where(current_vals == funcs[incorrect_positions[0]])[0])
#     print(incorrect_positions[0])
#     print(funcs)
#     print(current_vals)
#     print(incorrect_positions)

In [655]:
genSwapCircuit(qc, current_vals, funcs)
qc.draw()

Current Position: [ 1 -1  3 -4 -5]
Intended Position: [-4.  3. -1. -5.  1.]
Current Position: [-4  3 -1 -5  1]
Intended Position: [-4.  3. -1. -5.  1.]


┌─────────────────┐                     
q_0: ┤ initialize(0,1) ├───────■────────X────
     ├─────────────────┤     ┌─┴─┐┌───┐ │    
q_1: ┤ initialize(1,0) ├─|0>─┤ X ├┤ X ├─┼──X─
     ├─────────────────┤     └───┘└───┘ │  │ 
q_2: ┤ initialize(1,0) ├────────────────┼──X─
     ├─────────────────┤┌───┐           │    
q_3: ┤ initialize(0,1) ├┤ X ├───────────X──X─
     ├─────────────────┤├───┤              │ 
q_4: ┤ initialize(1,0) ├┤ X ├──────────────X─
     └─────────────────┘└───┘